In [3]:
using NearestNeighbors
using WordEmbeddings, SoftmaxClassifier
using Utils
using Query
using Base.Collections
using WordStreams
using DataStructures
using Clustering
using Distances
using Iterators
using StatsBase
using Training

INFO: Recompiling stale cache file /home/ubuntu/.julia/lib/v0.5/Iterators.ji for module Iterators.
  likely near /mnt/phd/prototypes/SenseSplittingWord2Vec/src/WordDistributions.jl:2
  likely near /mnt/phd/prototypes/SenseSplittingWord2Vec/src/WordDistributions.jl:2
  likely near /mnt/phd/prototypes/SenseSplittingWord2Vec/src/WordDistributions.jl:2

Use "Dict(a=>b for (a,b) in c)" instead.


In [ ]:
ee = restore("models/ss/WestburyLab.wikicorp.201004_100__m70000000.model");
dtree,labels = nn_tree(ee)
""

In [10]:
@time find_nearest_words(ee,"Australia",dtree,labels )

 40.743184 seconds (136.65 M allocations: 3.976 GB, 34.01% gc time)


Array{Tuple{Tuple{String,Int64},Float32},1}[Tuple{Tuple{String,Int64},Float32}[(("Australia",1),0.0),(("Jasco",1),0.349752),(("Kii-Tanabe",1),0.350159),(("Porsha",1),0.352619),(("Djwhal",1),0.353735)],Tuple{Tuple{String,Int64},Float32}[(("Australia",2),0.000155425),(("14,862",1),0.350395),(("rutty",1),0.351318),(("Nappster",1),0.353773),(("hypobromite",1),0.353882)],Tuple{Tuple{String,Int64},Float32}[(("Australia",3),0.0),(("classification",12),0.354916),(("awards",17),0.355997),(("Fredy",1),0.357518),(("Darnall",1),0.358537)],Tuple{Tuple{String,Int64},Float32}[(("Australia",4),0.0),(("3–0",1),0.339689),(("Sinsitro",1),0.348005),(("Au-Haidhausen",1),0.348443),(("monoembryonic",1),0.354063)],Tuple{Tuple{String,Int64},Float32}[(("Australia",5),0.0),(("NAHL",1),0.347071),(("mull",1),0.354301),(("UEML",1),0.356421),(("Jadiel",1),0.357218)],Tuple{Tuple{String,Int64},Float32}[(("Australia",6),0.0),(("Ushba",1),0.356109),(("Aborigine",1),0.357822),(("shot",8),0.35835),(("Taróczy",1),0.361376)

In [35]:
typeof(ee.trained_times) |> println
typeof(ee.trained_times["so"]) |> println



DataStructures.DefaultDict{String,DataStructures.DefaultDict{Int64,Int64,F},WordEmbeddings.##4#6}
DataStructures.DefaultDict{Int64,Int64,WordEmbeddings.##5#7}


In [29]:
values(ee.trained_times) |> first |> values

Int64[]

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl
 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in broadcast at ./deprecated.jl:400 [inlined]
 in length at /home/ubuntu/.julia/v0.5/DataStructures/src/delegate.jl:11 [inlined]
 in length at ./dict.jl:183 [inlined]
 in _similar_for at ./array.jl:207 [inlined]
 in _collect at ./array.jl:227 [inlined]
 in collect at ./array.jl:223 [inlined]
 in show(::Base.AbstractIOBuffer{Array{UInt8,1}}, ::Base.ValueIterator{DataStructures.DefaultDict{Int64,Int64,WordEmbeddings.#

In [38]:
collect(values(first(values(ee.trained_times))))

Int64[]

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl


In [41]:
trained_times = maximum([length(values(wtt)) for wtt in  values(ee.trained_times)]);
trained_times

0

In [47]:
length(collect(filter(word->length(ee.embedding[word])>1,keys(ee.embedding))))

5023

In [42]:
length(ee.distribution)

1531453

In [ ]:
function load_wordsim353(filename="data/corpora/wordsim353/combined.csv")
    wordsims = readdlm(filename, ','; skipstart=1)
    wordpairs = convert(Matrix{String},wordsims[:,1:2])
    groundsim = convert(Vector{Float64},wordsims[:,3])
    (wordpairs, groundsim)
end

In [ ]:
function get_sims(ee::FixedWordSenseEmbedding, wordpairs::Matrix{String})
    sims = Vector{Float64}(size(wordpairs,1))
    for ii in 1:size(wordpairs,1)
        try
            wvs_1 = hcat(ee.embedding[wordpairs[ii,1]]...)
            wvs_2 = hcat(ee.embedding[wordpairs[ii,2]]...)
            sims[ii]=1-minimum(pairwise(CosineDist(), wvs_1,wvs_2))
        catch ex
            warn(ex)
            sims[ii]=NaN
        end
    end
    return sims
end
sims = get_sims(ee,map(lowercase,wordpairs))
corspearman(groundsim,sims)

In [ ]:
function load_SCWS(filename = "data/corpora/SCWS/ratings.txt")
    function get_context2(fulltext)
        Task() do
            skip_next = 0
            for word in split(fulltext, ' ')
                skip_next-=1
                if word == "<b>"
                    skip_next=3 
                end
                skip_next>0 && continue
                produce(lowercase(word))
            end
        end |> collect
    end
    entries = readdlm(filename, '\t', String; quotes=false, comments=false)
    wordpairs = map(lowercase,entries[:,[2,4]])
    groundsim = map(s->parse(Float64,s), entries[:,8])
    contexts = map(get_context2,entries[:,[6,7]])
    (wordpairs,contexts,groundsim,)
end
wordpairs,contexts,groundsim = load_SCWS();
""

In [ ]:
function get_sims(ee::FixedWordSenseEmbedding, wordpairs::Matrix{String}, contexts::Matrix)
    sims = Vector{Float64}(size(wordpairs,1))
    for ii in 1:size(wordpairs,1)
        try
            
            sense1 = Training.WSD(ee, wordpairs[ii,1],contexts[ii,1];skip_oov=true)
            wv1 = ee.embedding[wordpairs[ii,1]][sense1]
            
            sense2 = Training.WSD(ee, wordpairs[ii,2],contexts[ii,2];skip_oov=true)
            wv2 = ee.embedding[wordpairs[ii,2]][sense2]
            
            sims[ii]=1-cosine_dist(wv1,wv2)
        catch ex
            if typeof(ex)==KeyError
                warn(ex)
                sims[ii]=NaN
            else
                rethrow(ex)
            end
        end
    end
    return sims
end

In [ ]:
local_sims = get_sims(ee,wordpairs,contexts)

In [ ]:
corspearman(groundsim,local_sims)